In [6]:
# from pyspark import SparkContext
from os.path import abspath
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import StructType, StructField, IntegerType

# import findspark
# findspark.init()

In [17]:


# Create a SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("pyspark app test") \
    .enableHiveSupport() \
    .config("spark.jars", "/Users/hmakhlouf/Desktop/TechCnsltng_WorkSpace/config/postgresql-42.7.2.jar") \
    .config("hive.metastore.uris", "thrift://ip-172-31-3-80.eu-west-2.compute.internal:8889") \
    .getOrCreate()
  

In [3]:
## 1- Establish the connection to PostgresSQL and read data from the postgres Database -testdb
# PostgresSQL connection properties
postgres_url = "jdbc:postgresql://ec2-3-9-191-104.eu-west-2.compute.amazonaws.com:5432/testdb"
postgres_properties = {
    "user": "consultants",
    "password": "WelcomeItc@2022",
    "driver": "org.postgresql.Driver",
}
postgres_table_name = "car_insurance_claims"

In [4]:
df_postgres = spark.read.jdbc(url=postgres_url, table=postgres_table_name, properties=postgres_properties)
df_postgres.show(3)

24/03/13 16:46:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+--------+-------+----+--------+----+-------+-------+--------+-------+------+------------+-------------+--------+-------+--------+---+----------+-------+--------+--------+-------+-------+-------+-------+----------+--------------------+
|       ID|KIDSDRIV|  BIRTH| AGE|HOMEKIDS| YOJ| INCOME|PARENT1|HOME_VAL|MSTATUS|GENDER|   EDUCATION|   OCCUPATION|TRAVTIME|CAR_USE|BLUEBOOK|TIF|  CAR_TYPE|RED_CAR|OLDCLAIM|CLM_FREQ|REVOKED|MVR_PTS|CLM_AMT|CAR_AGE|CLAIM_FLAG|          URBANICITY|
+---------+--------+-------+----+--------+----+-------+-------+--------+-------+------+------------+-------------+--------+-------+--------+---+----------+-------+--------+--------+-------+-------+-------+-------+----------+--------------------+
|501699878|       0|31DEC58|40.0|       1|16.0|$66,987|     No|$236,897|    Yes|   z_F|   Bachelors|      Manager|      28|Private| $12,200| 13|Sports Car|     no|  $8,216|       3|     No|      6|     $0|   11.0|         0| Highly Urban/ Urban|
|624637727|     

In [8]:
# Rename column from "ID" to "policy_number"
df_postgres = df_postgres.withColumnRenamed("ID", "POLICY_NUMBER")
df_postgres.printSchema()





root
 |-- POLICY_NUMBER: long (nullable = true)
 |-- KIDSDRIV: long (nullable = true)
 |-- BIRTH: string (nullable = true)
 |-- AGE: double (nullable = true)
 |-- HOMEKIDS: long (nullable = true)
 |-- YOJ: double (nullable = true)
 |-- INCOME: string (nullable = true)
 |-- PARENT1: string (nullable = true)
 |-- HOME_VAL: string (nullable = true)
 |-- MSTATUS: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- EDUCATION: string (nullable = true)
 |-- OCCUPATION: string (nullable = true)
 |-- TRAVTIME: long (nullable = true)
 |-- CAR_USE: string (nullable = true)
 |-- BLUEBOOK: string (nullable = true)
 |-- TIF: long (nullable = true)
 |-- CAR_TYPE: string (nullable = true)
 |-- RED_CAR: string (nullable = true)
 |-- OLDCLAIM: string (nullable = true)
 |-- CLM_FREQ: long (nullable = true)
 |-- REVOKED: string (nullable = true)
 |-- MVR_PTS: long (nullable = true)
 |-- CLM_AMT: string (nullable = true)
 |-- CAR_AGE: double (nullable = true)
 |-- CLAIM_FLAG: long (nullable

In [10]:
# Specify the column to be modified
columns_to_modify = ["MSTATUS", "GENDER", "EDUCATION", "OCCUPATION", "CAR_TYPE", "URBANICITY"]
# Modify string values by removing "z_"
for column in columns_to_modify:
    df_postgres = df_postgres.withColumn(column, regexp_replace(col(column), "^z_", ""))

df_postgres.show(3)

+-------------+--------+-------+----+--------+----+-------+-------+--------+-------+------+------------+-----------+--------+-------+--------+---+----------+-------+--------+--------+-------+-------+-------+-------+----------+-------------------+
|POLICY_NUMBER|KIDSDRIV|  BIRTH| AGE|HOMEKIDS| YOJ| INCOME|PARENT1|HOME_VAL|MSTATUS|GENDER|   EDUCATION| OCCUPATION|TRAVTIME|CAR_USE|BLUEBOOK|TIF|  CAR_TYPE|RED_CAR|OLDCLAIM|CLM_FREQ|REVOKED|MVR_PTS|CLM_AMT|CAR_AGE|CLAIM_FLAG|         URBANICITY|
+-------------+--------+-------+----+--------+----+-------+-------+--------+-------+------+------------+-----------+--------+-------+--------+---+----------+-------+--------+--------+-------+-------+-------+-------+----------+-------------------+
|    501699878|       0|31DEC58|40.0|       1|16.0|$66,987|     No|$236,897|    Yes|     F|   Bachelors|    Manager|      28|Private| $12,200| 13|Sports Car|     no|  $8,216|       3|     No|      6|     $0|   11.0|         0|Highly Urban/ Urban|
|    6246377

In [19]:
df_postgres.count()

8754

In [36]:
## 2. load df_postgres to hive table
# Create database
spark.sql("CREATE DATABASE IF NOT EXISTS hocinedb")

DataFrame[]

In [38]:
 #Hive database and table names
hive_database_name = "hocinedb"
hive_table_name = "carInsurance"

# Create Hive Internal table over project1db
df_postgres.write.mode('overwrite').saveAsTable(hocinedb.carinsuaranceclaim)

# Read Hive table
df = spark.read.table(hocinedb.carinsuaranceclaim)
df.show(2)



NameError: name 'hocinedb' is not defined

In [44]:
existing_hive_data = spark.read.table("{}.{}".table("project1db.carinsuranceclaims")
existing_hive_data.show(3)

SyntaxError: invalid syntax (4283840756.py, line 2)